In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

import IPython
import requests
import dotenv
import pandas as pd
import numpy as np

from backend.location import Location, make_grid, spherical_distance
from backend import gmaps

dotenv.load_dotenv()

In [ ]:
GMAPS_API_KEY = os.environ["GMAPS_API_KEY"]

In [ ]:
# zurich = Location(47.378, 8.540)
zurich = Location(47.369, 8.5415)
# zurich = Location(45, 130)
zoom = 15
grid = make_grid(zurich, zoom, size=9)

image = gmaps.get_static_map(zurich, zoom, markers=[])

IPython.display.Image(image)

In [ ]:
matrix = list(gmaps.get_distance_matrix(grid, grid))

In [ ]:
matrix

In [ ]:
len(matrix)

In [ ]:
with open("9x9matrix-v2.json", "w") as f:
    json.dump(matrix, f)

In [ ]:
with open("9x9matrix-v2-request.json", "w") as f:
    json.dump(gmaps.get_distance_matrix_api_payload(grid, grid), f)

In [ ]:
matrix

In [ ]:
[x for x in matrix if "destinationIndex" not in x]

In [ ]:
x

## Analysis

In [ ]:
with open("../frontend/src/assets/9x9matrix-v2.json") as f:
    matrix = json.load(f)

In [ ]:
df = pd.DataFrame(matrix)
df = df.set_index(["originIndex", "destinationIndex"]).sort_index()

In [ ]:
df["durationSec"] = df["duration"].apply(lambda x: x if pd.isna(x) else int(x[:-1]))

In [ ]:
df["originLat"] = df.index.get_level_values(0).map(lambda i: grid[i].lat)
df["originLng"] = df.index.get_level_values(0).map(lambda i: grid[i].lng)
df["destinationLat"] = df.index.get_level_values(1).map(lambda i: grid[i].lat)
df["destinationLng"] = df.index.get_level_values(1).map(lambda i: grid[i].lng)

In [ ]:
df

In [ ]:
df["sphericalDistance"] = df.apply(
    lambda row: spherical_distance(
        Location(row.originLat, row.originLng),
        Location(row.destinationLat, row.destinationLng),
    ),
    axis=1,
)

In [ ]:
df["distanceRatio"] =  df["sphericalDistance"] / df["distanceMeters"]
df["timeRatio"] =  df["sphericalDistance"] / df["durationSec"]

In [ ]:
df.sort_values("distanceRatio", ascending=False)

In [ ]:
import plotly.express as px

In [ ]:
px.histogram(df, x="timeRatio")

In [ ]:
px.scatter(df, "distanceRatio", "timeRatio")

In [ ]:
px.histogram(df, x="distanceRatio")

In [ ]:
df

In [ ]:
df["velocity"] = df["distanceMeters"] / df["duration_sec"]

In [ ]:
df.query("~velocity.isna()").sort_values("velocity", ascending=False)